## Data Wrangling

In [1]:
#load python packages
import os
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import stem

In [2]:
os.getcwd()

'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two'

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'Capstone Two',
 'data',
 'EDA_Quora Insincere Capstone Data.ipynb',
 'embeddings',
 'figures',
 'models',
 'Preprocessing and Training Data Development.ipynb',
 'Quora Insincere Capstone Data Wrangling.ipynb',
 'Quora Insincere Classification - Modeling Step.ipynb',
 'ROC-AUC score.png',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'Untitled.ipynb']

In [4]:
#path=""
#os.chdir(path)

### Data Collection 

#### Load train and test csv files

In [5]:
file_name_1 = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two\\train.csv'
file_name_2 = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two\\test.csv'

In [6]:
# Create train and test dataframes by reading the csv files
train = pd.read_csv(file_name_1)
test = pd.read_csv(file_name_2)

In [7]:
#Concat both train and test files for initial pre-processing
quora_df = pd.concat([train, test])

In [8]:
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0


In [9]:
#Number of texts retrieved from merged dataframe
quora_df.shape[0]

1424381

### Data Organization

In [10]:
path = 'C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two'

In [11]:
print ("The current working directory is %s" % path)

The current working directory is C:\Shalin\Springboard bootcamp\projects\Capstone project 2\Capstone Two


#### Create subfolders in your project directory

In [12]:
#data = os.makedirs('data')

In [13]:
#figures = os.makedirs('figures')

In [14]:
#models = os.makedirs('models')

In [15]:
os.listdir()

['.ipynb_checkpoints',
 'Capstone Two',
 'data',
 'EDA_Quora Insincere Capstone Data.ipynb',
 'embeddings',
 'figures',
 'models',
 'Preprocessing and Training Data Development.ipynb',
 'Quora Insincere Capstone Data Wrangling.ipynb',
 'Quora Insincere Classification - Modeling Step.ipynb',
 'ROC-AUC score.png',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'Untitled.ipynb']

### Data Definition

In [16]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1424381 entries, 0 to 375805
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   qid            1424381 non-null  object 
 1   question_text  1424381 non-null  object 
 2   target         1048575 non-null  float64
dtypes: float64(1), object(2)
memory usage: 43.5+ MB


In [17]:
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0


In [18]:
quora_df.tail(2)

,qid,question_text,target
375804,ffffb1f7f1a008620287,What are the causes of refraction of light?,NaN
375805,fffff85473f4699474b0,Climate change is a worrying topic. How much t...,NaN


In [19]:
quora_df.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [20]:
quora_df.dtypes

qid               object
question_text     object
target           float64
dtype: object

#### Data cleaning for target column

In [21]:
quora_df['target'].isna().sum()

375806

In [22]:
quora_df = quora_df[quora_df['target'].notna()]
quora_df.tail(2)

,qid,question_text,target
1048573,cd763d37d3d7cfcd42ce,What is the difference between the old currenc...,0.0
1048574,cd7642554d107f946d8a,What is the full form of DML?,0.0


In [23]:
quora_df['target'].isna().sum()

0

#### Convert target column into 'integer' since the values will either be 0 or 1 (prediction for sensitive vs insensitive data)

In [24]:
quora_df['target'] = quora_df['target'].apply(np.int64)

In [25]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1048575 non-null  object
 1   question_text  1048575 non-null  object
 2   target         1048575 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


In [26]:
quora_df.describe()

,target
count,1.048575e+06
mean,6.177336e-02
std,2.407436e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,1.000000e+00


**Extract only the question_text column from the dataframe to perform further data cleaning**

In [27]:
quora_question_df = quora_df['question_text']
quora_question_df.head(2)

0    How did Quebec nationalists see their province...
1    Do you have an adopted dog, how would you enco...
Name: question_text, dtype: object

**Find max and min length of question from the list**

In [28]:
max_len = 0
for length in quora_question_df: 
    if len(length) > max_len: 
        max_len = len(length) 
        question = length 
print("Maximum length string is : " + question) 

Maximum length string is : What is [math]\frac{\int_{1x^5}^{3x^{-5}} \tan(\tan({\boxed{\int_{1x^0}^{1x^2} \sum_{\varpi=1}^{\infty} \int_{2x^{-3}}^{2x^2} \sum_{\alpha=7}^{\infty} \underbrace{\sqrt[2]{1x^5}}_{\text{Gauss's Law of Theoretical Probability.}} d\tau dx}}^{1x^0})) d\mu}{\int_{2x^{-3}}^{1x^5} \cos(\int_{2x^2}^{1x^{-3}} \frac{\sqrt[2]{\overbrace{\underbrace{\frac{3x^3+3x^5}{\sqrt[3]{2x^{-3}}}}_{\text{Gauss's Law of Theoretical Probability.}} \times \overbrace{\tan(2x^0)}^{\text{Gauss's Law of Theoretical Probability.}}-\sum_{4=7}^{\infty} \boxed{3x^{-5}}}^{\text{Inverse Function.}}}}{{\boxed{\int_{2x^2}^{2x^4} 3x^1 d9} \div \sum_{6=6}^{\infty} \sqrt[3]{2x^2}+\sqrt[4]{\sin(2x^0+3x^0)}}^{2x^{-4}}+\boxed{\frac{\vec{\boxed{\sum_{\gamma=10}^{\infty} 1x^{-5}}}}{\frac{\sum_{\iota=2}^{\infty} 1x^{-5}-\frac{3x^{-1}}{1x^{-4}}}{\sin(\tan(3x^{-2}))}}}} \times \boxed{\sqrt[2]{{{{\sqrt[5]{2x^5}}^{2x^{-1}}}^{2x^{-1}} \div \sum_{\chi=6}^{\infty} \int_{1x^4}^{2x^{-4}} 3x^2 d\vartheta+{2x^{-3}}^

**Alternative way of finding the max length of question from the list**

In [29]:
question_max = max(quora_question_df, key = len) 
question_max

"What is [math]\\frac{\\int_{1x^5}^{3x^{-5}} \\tan(\\tan({\\boxed{\\int_{1x^0}^{1x^2} \\sum_{\\varpi=1}^{\\infty} \\int_{2x^{-3}}^{2x^2} \\sum_{\\alpha=7}^{\\infty} \\underbrace{\\sqrt[2]{1x^5}}_{\\text{Gauss's Law of Theoretical Probability.}} d\\tau dx}}^{1x^0})) d\\mu}{\\int_{2x^{-3}}^{1x^5} \\cos(\\int_{2x^2}^{1x^{-3}} \\frac{\\sqrt[2]{\\overbrace{\\underbrace{\\frac{3x^3+3x^5}{\\sqrt[3]{2x^{-3}}}}_{\\text{Gauss's Law of Theoretical Probability.}} \\times \\overbrace{\\tan(2x^0)}^{\\text{Gauss's Law of Theoretical Probability.}}-\\sum_{4=7}^{\\infty} \\boxed{3x^{-5}}}^{\\text{Inverse Function.}}}}{{\\boxed{\\int_{2x^2}^{2x^4} 3x^1 d9} \\div \\sum_{6=6}^{\\infty} \\sqrt[3]{2x^2}+\\sqrt[4]{\\sin(2x^0+3x^0)}}^{2x^{-4}}+\\boxed{\\frac{\\vec{\\boxed{\\sum_{\\gamma=10}^{\\infty} 1x^{-5}}}}{\\frac{\\sum_{\\iota=2}^{\\infty} 1x^{-5}-\\frac{3x^{-1}}{1x^{-4}}}{\\sin(\\tan(3x^{-2}))}}}} \\times \\boxed{\\sqrt[2]{{{{\\sqrt[5]{2x^5}}^{2x^{-1}}}^{2x^{-1}} \\div \\sum_{\\chi=6}^{\\infty} \\int_{1

In [30]:
question_min = min(quora_question_df, key = len) 
question_min

'"'

### Data Cleaning

As identified in earlier step during data definition step, null values were already removed from 'target' column of the dataframe. This data cleaning will be performed only on the 'question_text' column of dataframe

**Convert text to lowercase**

In [31]:
quora_df['question_text'] = quora_df['question_text'].str.lower()
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,"do you have an adopted dog, how would you enco...",0


**Remove punctuation**
The following code removes this set of symbols [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:

In [32]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(string.punctuation)])
    return s

quora_df['question_text'] = quora_df['question_text'].apply(remove_punctuation)

In [33]:
# Seems like quotes are not removed from the dataset. Hence work on removing them
quora_df['question_text']= quora_df['question_text'].str.replace('’', '')
quora_df['question_text']= quora_df['question_text'].str.replace('?', '')

In [34]:
quora_question_df = quora_df['question_text']
quora_question_df.head(2)

0    how did quebec nationalists see their province...
1    do you have an adopted dog how would you encou...
Name: question_text, dtype: object

There are whitespaces, new line characters which can be removed (on deriving length and min length strings)

In [35]:
question_max = max(quora_question_df, key = len) 
question_max

'in star trek 2013 why did they \n\nspoilers\nspoilers\nspoilers\nspoilers\n\n1make warping look quite a bit like an hyperspace jump\n2what in the world were those bright particles as soon as they jumped\n3why in the world did they make it possible for two entities to react in warp space in separate jumps\n4why did spock get emotions for this movie\n5what was the point of hiding the enterprise underwater\n6when they were intercepted by the dark ship how come they reached earth when they were far away from heri dont seem to remember the scene where they warp to earth\n7how did the ship enter earths atmosphere when it wasnt even in orbit\n8when scotty opened the door of the black ship  how come pike and khan didnt slow down'

In [36]:
question_min = min(quora_question_df, key = len) 
question_min

''

**Remove newline characters and whitespaces**

- Remove newline characters

In [37]:
quora_df['question_text']= [re.sub(r'\n\d', '?', text).replace("\nspoilers","").replace("\n","") for text in quora_df['question_text']]
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0


In [38]:
quora_question_df = quora_df['question_text']
quora_question_df.head(2)

0    how did quebec nationalists see their province...
1    do you have an adopted dog how would you encou...
Name: question_text, dtype: object

In [39]:
question_max = max(quora_question_df, key = len) 
question_max

'in star trek 2013 why did they ?make warping look quite a bit like an hyperspace jump?what in the world were those bright particles as soon as they jumped?why in the world did they make it possible for two entities to react in warp space in separate jumps?why did spock get emotions for this movie?what was the point of hiding the enterprise underwater?when they were intercepted by the dark ship how come they reached earth when they were far away from heri dont seem to remember the scene where they warp to earth?how did the ship enter earths atmosphere when it wasnt even in orbit?when scotty opened the door of the black ship  how come pike and khan didnt slow down'

- Remove whitespaces : to remove leading and ending spaces, you can use the strip() function:

In [40]:
quora_df['question_text'] = [x.strip() for x in quora_df['question_text']]
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0


#### Handling missing values in case of empty string values

In [41]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1048575 non-null  object
 1   question_text  1048575 non-null  object
 2   target         1048575 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


In [42]:
# Find out if there are any missing question_text values
null_df = [text for text in quora_df['question_text'] if len(text) < 1]
null_df

['']

In [43]:
null_df = np.where(quora_df['question_text'] == '')
null_df

(array([420816], dtype=int64),)

In [44]:
quora_df.iloc[420816]

qid              527aac2ce6f12f789fe5
question_text                        
target                              1
Name: 420816, dtype: object

In [45]:
quora_df = quora_df.drop(quora_df.index[420816])
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048574 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   qid            1048574 non-null  object
 1   question_text  1048574 non-null  object
 2   target         1048574 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


**Remove stop words**
- “Stop words” are the most common words in a language like “the”, “a”, “on”, “is”, “all”. These words do not carry important meaning and are usually removed from texts. It is possible to remove stop words using Natural Language Toolkit (NLTK), a suite of libraries and programs for symbolic and statistical natural language processing.

In [46]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gosal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gosal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
quora_df.head(2)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0


In [48]:
stop_words = stopwords.words('english')

quora_df['questions_without_stopwords'] = quora_df['question_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
quora_df.head(2)

,qid,question_text,target,questions_without_stopwords
0,00002165364db923c7e6,how did quebec nationalists see their province...,0,quebec nationalists see province nation 1960s
1,000032939017120e6e44,do you have an adopted dog how would you encou...,0,adopted dog would encourage people adopt shop


In [49]:
del quora_df['question_text']

In [50]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048574 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   qid                          1048574 non-null  object
 1   target                       1048574 non-null  int64 
 2   questions_without_stopwords  1048574 non-null  object
dtypes: int64(1), object(2)
memory usage: 32.0+ MB


**Stemming using NLTK**
- Stemming is a process of reducing words to their word stem, base or root form (for example, books — book, looked — look). The main two algorithms are Porter stemming algorithm (removes common morphological and inflexional endings from words [14]) and Lancaster stemming algorithm (a more aggressive stemming algorithm). In the “Stemming” sheet of the table some stemmers are described

In [51]:
stemmer=PorterStemmer()

quora_df['question_stemmed'] = quora_df['questions_without_stopwords'].apply(lambda x: stemmer.stem(x)) # Stem every word.
quora_df = quora_df.drop(columns=['questions_without_stopwords']) # Get rid of the unstemmed column.
quora_df.head(2) # Print dataframe.

,qid,target,question_stemmed
0,00002165364db923c7e6,0,quebec nationalists see province nation 1960
1,000032939017120e6e44,0,adopted dog would encourage people adopt shop


In [52]:
# Perform stemming again to ensure all words are properly stemmed. 
from nltk.stem import PorterStemmer
import stem

porter_stemmer = PorterStemmer()

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

quora_df['question_stemmed'] = quora_df['question_stemmed'].apply(stem_sentences)
#quora_df = quora_df.drop(columns=['question_stemmed']) # Get rid of the redundant column.
quora_df.head(5)

,qid,target,question_stemmed
0,00002165364db923c7e6,0,quebec nationalist see provinc nation 1960
1,000032939017120e6e44,0,adopt dog would encourag peopl adopt shop
2,0000412ca6e4628ce2cf,0,veloc affect time veloc affect space geometri
3,000042bf85aa498cd78e,0,otto von guerick use magdeburg hemisph
4,0000455dfa3e01eae3af,0,convert montra helicon mountain bike chang tyr


#### Add columns to include a) length of each question b) # of words in the dataframe (for any future exercise)

In [53]:
quora_df['question_length']  = quora_df['question_stemmed'].str.len()
quora_df['total_words'] = [len(x.split()) for x in quora_df['question_stemmed'].tolist()]
quora_df.head(2)

,qid,target,question_stemmed,question_length,total_words
0,00002165364db923c7e6,0,quebec nationalist see provinc nation 1960,42,6
1,000032939017120e6e44,0,adopt dog would encourag peopl adopt shop,41,7


In [54]:
quora_df.sort_values(by='question_length', ascending=False, inplace=True)
quora_df = quora_df[['qid', 'question_stemmed', 'target', 'question_length', 'total_words']]
quora_df.head(10)

,qid,question_stemmed,target,question_length,total_words
443216,56da6b6875d686b48fde,mathfracint1x53x5 tantanboxedint1x01x2 sumvarp...,1,566,40
163583,1ffca149bd0a19cd714c,mathoverbracesumvartheta8infti vecfracsumkappa...,1,494,31
522266,663c7523d48f5ee66a3e,star trek 2013 ?make warp look quit bit like h...,0,419,66
341312,42dc38e62f0bdebfa6b6,answer mathfrac4222262mid 5501int846fracomega2...,0,332,11
149952,1d53c9c017999b4f77e2,8430397824532987451912384179815150754023741609...,0,241,3
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,212,32
158221,1ef37766a923f24f2182,didnt genghi khan thee great khan abolish dest...,1,209,32
393959,4d2e2796dd1ced2c8e64,prospect intern student httpwwwophammainquorac...,0,209,12
753959,93b9389889ad4aa8518a,waz genghiz khanz thee great khanz go get libe...,1,207,32
251508,313a5512fefd7dbf0fba,safe north east indian assam arunach pradesh m...,0,204,27


### Data cleaning continued

#### Removing entries that have 0 'question length'

In [55]:
quora_df = quora_df[quora_df['question_length'] != 0]
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048523 entries, 443216 to 69097
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   qid               1048523 non-null  object
 1   question_stemmed  1048523 non-null  object
 2   target            1048523 non-null  int64 
 3   question_length   1048523 non-null  int64 
 4   total_words       1048523 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 48.0+ MB


As you could see above, more than 50 rows were removed since there was no text associated to them. In an earlier cleaning step, there was an effort to remove entries with missing values with text length < 1, but that didnt remove everything (maybe those were whitespaces yet didnt account for len(text)<1)

In [56]:
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
443216,56da6b6875d686b48fde,mathfracint1x53x5 tantanboxedint1x01x2 sumvarp...,1,566,40
163583,1ffca149bd0a19cd714c,mathoverbracesumvartheta8infti vecfracsumkappa...,1,494,31
522266,663c7523d48f5ee66a3e,star trek 2013 ?make warp look quit bit like h...,0,419,66
341312,42dc38e62f0bdebfa6b6,answer mathfrac4222262mid 5501int846fracomega2...,0,332,11
149952,1d53c9c017999b4f77e2,8430397824532987451912384179815150754023741609...,0,241,3


#### Removing rows with very long words (likely junk values) 
- As seen above, there are still many records that have a very long character length and can be removed (we can choose 15 characters)
- Also alphanumeric values (that have mix of numbers of strings) can be filtered out
- However, need to be wary of the fact that since punctuations were removed, we may have certain rows with http OR www values AND could have a mix of numbers and characters AND still could exceed the threshold word length (as described in the above point) that we are looking to filter, hence they need to be retained 

In [57]:
quora_df = quora_df[~quora_df['question_stemmed'].str.contains(r'(?<!\S)(?!http|\S*www\.)\S{15}')]

In [58]:
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,212,32
158221,1ef37766a923f24f2182,didnt genghi khan thee great khan abolish dest...,1,209,32
393959,4d2e2796dd1ced2c8e64,prospect intern student httpwwwophammainquorac...,0,209,12
753959,93b9389889ad4aa8518a,waz genghiz khanz thee great khanz go get libe...,1,207,32
251508,313a5512fefd7dbf0fba,safe north east indian assam arunach pradesh m...,0,204,27


In [59]:
quora_df.tail(5)

,qid,question_stemmed,target,question_length,total_words
395648,4d836aebed79b49fbaab,n,0,1,1
533374,68707bcc73085d438385,n,0,1,1
894300,af37c56bf566734dc780,c,0,1,1
505275,62ef775bc180cc2956d0,9,0,1,1
69097,0d8c9f2ac8285c2e5070,b,0,1,1


- Find count of all question_text grouping to see the question_length

In [60]:
question_length_grouping = quora_df.groupby('question_length').question_stemmed.nunique().head(15)
question_length_grouping

question_length
1        10
2        88
3       354
4       733
5       848
6      1076
7      1822
8      3345
9      4969
10     6238
11     7212
12     8587
13    10420
14    12716
15    15031
Name: question_stemmed, dtype: int64

In [61]:
char_len_9 = quora_df[quora_df['question_length'] == 9]
char_len_9.tail(5)

,qid,question_stemmed,target,question_length,total_words
282522,374ead9ee920cdf8b550,use trust,0,9,2
603151,761fff772848d03f3494,maxin wat,0,9,2
863771,a940f47a642e7617b3b6,radiu bon,0,9,2
196493,2669c0d6405cfbddd7f6,bias forg,0,9,2
358819,465383d7c245537e0296,hill form,0,9,2


In [62]:
char_len_10 = quora_df[quora_df['question_length'] == 10]
char_len_10.tail(5)

,qid,question_stemmed,target,question_length,total_words
353917,455d421ac9d9eb7e1903,dumb idiot,0,10,2
917262,b3bbb7e84b0f71a4a837,fe2o3 poss,0,10,2
855614,a7a3984ffc9e3d64ccef,curs bulli,0,10,2
847366,a60aad66b617675722ad,what plast,0,10,2
656843,80a92e2776e2aa8a6e41,fact scari,0,10,2


#### Remove rows where length of text is < 10
- In earlier section, we removed all null text rows based on 'len' method
- However based on the above grouping, it seems like it might be worth to remove all rows where question length is <10. And also worth to remove rows that have just 1 word 

In [63]:
quora_df = quora_df[quora_df['question_length'] >= 10]
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,212,32
158221,1ef37766a923f24f2182,didnt genghi khan thee great khan abolish dest...,1,209,32
393959,4d2e2796dd1ced2c8e64,prospect intern student httpwwwophammainquorac...,0,209,12
753959,93b9389889ad4aa8518a,waz genghiz khanz thee great khanz go get libe...,1,207,32
251508,313a5512fefd7dbf0fba,safe north east indian assam arunach pradesh m...,0,204,27


In [64]:
quora_df = quora_df[quora_df['total_words'] > 1]
quora_df.head(5)

,qid,question_stemmed,target,question_length,total_words
460851,5a3f6882c1fc8319eff1,didnt genghiz khan thee great khan abolish des...,1,212,32
158221,1ef37766a923f24f2182,didnt genghi khan thee great khan abolish dest...,1,209,32
393959,4d2e2796dd1ced2c8e64,prospect intern student httpwwwophammainquorac...,0,209,12
753959,93b9389889ad4aa8518a,waz genghiz khanz thee great khanz go get libe...,1,207,32
251508,313a5512fefd7dbf0fba,safe north east indian assam arunach pradesh m...,0,204,27


In [65]:
quora_df.tail(5)

,qid,question_stemmed,target,question_length,total_words
353917,455d421ac9d9eb7e1903,dumb idiot,0,10,2
917262,b3bbb7e84b0f71a4a837,fe2o3 poss,0,10,2
855614,a7a3984ffc9e3d64ccef,curs bulli,0,10,2
847366,a60aad66b617675722ad,what plast,0,10,2
656843,80a92e2776e2aa8a6e41,fact scari,0,10,2


In [66]:
quora_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024881 entries, 460851 to 656843
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   qid               1024881 non-null  object
 1   question_stemmed  1024881 non-null  object
 2   target            1024881 non-null  int64 
 3   question_length   1024881 non-null  int64 
 4   total_words       1024881 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 46.9+ MB


In [67]:
quora_df.to_csv('C:\\Shalin\\Springboard bootcamp\\projects\\Capstone project 2\\Capstone Two\\data\\Quora_cleaned.csv')